Load packages

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from functions import *

Read data

In [17]:
df = read_score_df()
df.head()

,ID,USER_ID,NAME,DATETIME_START,DATE_START,TIME_START,EXERCISE_IDX,TAFEL,RAND_NUM,USER_ANSWER,SCORE,DURATION_TIME,MODEL_SCORE,PROBABILITY,CREATED_AT
0,256,None,Mama,2025-10-20T12:04:22+00:00,2025-10-20,12:04:22,1,2,3,6,1.0,1.933,0.517331,0.000000,2025-10-20T10:04:24.524953+00:00
1,255,None,Mama,2025-10-20T12:02:07+00:00,2025-10-20,12:02:07,1,2,9,18,1.0,1.992,0.502008,0.000000,2025-10-20T10:02:14.137267+00:00
2,147,None,Mama,2025-10-19T21:10:12+00:00,2025-10-19,21:10:12,2,9,9,81,1.0,2.284,0.437828,0.001773,2025-10-19T20:39:51.896924+00:00
3,129,None,Mama,2025-10-19T21:10:12+00:00,2025-10-19,21:10:12,1,7,7,49,1.0,2.876,0.347705,0.002233,2025-10-19T20:39:51.896924+00:00
4,154,None,Mama,2025-10-19T21:04:56+00:00,2025-10-19,21:04:55,2,5,6,30,1.0,2.070,0.483092,0.001607,2025-10-19T20:39:51.896924+00:00


In [ ]:
df.columns = [col.lower() for col in df.columns]
df = df.copy()
df["date_start"] = pd.to_datetime(df["date_start"], errors="coerce").dt.strftime("%Y-%m-%d")
df["date_start"].head()

In [ ]:
df["time_start"] = pd.to_datetime(df["time_start"].astype(str), errors="coerce").dt.strftime("%H:%M:%S")
df["time_start"].head()

Manipulate data


In [22]:
# transform SCORE column to 0 if score is 0.1
df["SCORE"] = df["SCORE"].apply(lambda x: 0 if x == 0.1 else x)
df["SCORE"]

0      1.0
1      1.0
2      1.0
3      1.0
4      1.0
      ... 
249    1.0
250    1.0
251    1.0
252    0.0
253    0.0
Name: SCORE, Length: 254, dtype: float64

In [36]:
df_per_day = (
    df.groupby(["DATE_START"], as_index=False)
    .agg(SEC_PER_DAG=("DURATION_TIME", "sum"), SCORE_PER_DAG=("SCORE", "mean"))
    .assign(MIN_PER_DAG=lambda x: x["SEC_PER_DAG"] / 60)
    .assign(SCORE_PER_DAG=lambda x: x["SCORE_PER_DAG"].round(2) * 100)
)
df_per_day.head()
df_calendar = (
    df_per_day.assign(DATE_START=pd.to_datetime(df_per_day["DATE_START"]))
    .assign(WEEKDAY=lambda x: x["DATE_START"].dt.day_name())
    .assign(
        TEXT=lambda x: x["DATE_START"].dt.strftime("%Y-%m-%d")
        + "\n[Timer: "
        + x["MIN_PER_DAG"].apply(lambda y: str(int(np.ceil(y))))
        + " min" + ", score: "+ x["SCORE_PER_DAG"].apply(lambda y: str(int(y))) + "%]")
    .assign(WEEK=lambda x: x["DATE_START"].dt.isocalendar().week)
    .groupby(["WEEK", "WEEKDAY"], as_index=False)
    .agg(TEXT=("TEXT", "first"))
    .pivot(index="WEEK", columns="WEEKDAY", values="TEXT")
)
df_calendar.head()

WEEKDAY,Monday,Sunday
WEEK,,
42,NaN,"2025-10-19\n[Timer: 15 min, score: 97%]"
43,"2025-10-20\n[Timer: 1 min, score: 100%]",NaN


Database manipulations


In [ ]:
url = st.secrets["SUPABASE_URL"]
key = st.secrets["SUPABASE_ANON_KEY"]
sb = create_client(url, key)

In [ ]:
# take only last row from df
last_row = df.tail(1)
last_row


In [39]:
import re
text="2025-10-20\n[Timer: 0 min, score: 0%]"
minutes = int(re.search(r'Timer:\s*(\d+)\s*min', text).group(1))
print(minutes) 

0


In [ ]:
save_score_df(sb, last_row)